In [1]:

! pip3 install -r requirements.txt


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 MB 31.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 KB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 4.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 75.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 114.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.9/748.9 KB 80.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 106.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 95.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.

In [2]:
import sys
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import numpy as np
# from multiprocessing import Pool  # , Process
from cerebros.simplecerebrosrandomsearch.simple_cerebros_random_search\
    import SimpleCerebrosRandomSearch
import pendulum
import pandas as pd
import tensorflow as tf
from cerebros.units.units import DenseUnit
from cerebros.denseautomlstructuralcomponent.dense_automl_structural_component\
    import zero_7_exp_decay, zero_95_exp_decay, simple_sigmoid
from ast import literal_eval

2023-01-09 06:03:14.664523: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-09 06:03:15.564873: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-09 06:03:15.565008: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-09 06:03:15.565021: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed proper

In [2]:

activation = 'relu'
predecessor_level_connection_affinity_factor_first = 2.0
predecessor_level_connection_affinity_factor_main = 0.97
max_consecutive_lateral_connections = 5
p_lateral_connection = 0.97
num_lateral_connection_tries_per_unit = 2
learning_rate = 0.001
epochs = 10 # [1, 100]
batch_size = 100
maximum_levels = 6 # [3,7]
maximum_units_per_level = 4 # [2,10]
maximum_neurons_per_unit = 4 # [2,20]


In [3]:

raw_data = pd.read_csv('diabetes.csv')
label = raw_data.pop('Outcome').astype(int).values

data_numeric = raw_data.astype(float)


data_np = data_numeric.values

# np.save('label', label, allow_pickle=True)
# np.save('data', data_np, allow_pickle=True)

TIME = pendulum.now(tz='America/New_York').__str__()[:16]\
    .replace('T', '_')\
    .replace(':', '_')\
    .replace('-', '_')
PROJECT_NAME = f'{TIME}_cerebros_auto_ml_test'

# data_np = np.load('data.npy', allow_pickle=True)
# labels = np.load('labels.npy', allow_pickle=True)

tensor_x =\
    tf.constant(data_np)

training_x = [tensor_x]

INPUT_SHAPES = [training_x[i].shape[1] for i in np.arange(len(training_x))]


#alcohol_labels = tf.constant(pd.get_dummies(pd.cut(white['alcohol'], np.arange(
#    white['alcohol'].min(), white['alcohol'].max()))).values)
# quality_labels = tf.constant(pd.get_dummies(white['quality']).values)

train_labels = [label]

# OUTPUT_SHAPES = [1]  # [train_labels[i].shape[1]
# for i in np.arange(len(train_labels))]


In [4]:


INPUT_SHAPES = [training_x[i].shape[1] for i in np.arange(len(training_x))]

OUTPUT_SHAPES = [1]

meta_trial_number = str(int(np.random.random() * 10 ** 12))


cerebros_automl = SimpleCerebrosRandomSearch(
    unit_type=DenseUnit,
    input_shapes=INPUT_SHAPES,
    output_shapes=OUTPUT_SHAPES,
    training_data=training_x,
    labels=train_labels,
    validation_split=0.35,
    direction='maximize',
    metric_to_rank_by="val_binary_accuracy",
    minimum_levels=2,
    maximum_levels=maximum_levels,
    minimum_units_per_level=1,
    maximum_units_per_level=maximum_units_per_level,
    minimum_neurons_per_unit=1,
    maximum_neurons_per_unit=maximum_neurons_per_unit,
    activation=activation,
    final_activation='sigmoid',
    number_of_architecture_moities_to_try=7,
    number_of_tries_per_architecture_moity=1,
    minimum_skip_connection_depth=1,
    maximum_skip_connection_depth=7,
    predecessor_level_connection_affinity_factor_first=predecessor_level_connection_affinity_factor_first,
    predecessor_level_connection_affinity_factor_first_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_main=predecessor_level_connection_affinity_factor_main,
    predecessor_level_connection_affinity_factor_main_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_decay_main=zero_7_exp_decay,
    seed=8675309,
    max_consecutive_lateral_connections=max_consecutive_lateral_connections,
    gate_after_n_lateral_connections=3,
    gate_activation_function=simple_sigmoid,
    p_lateral_connection=p_lateral_connection,
    p_lateral_connection_decay=zero_95_exp_decay,
    num_lateral_connection_tries_per_unit=num_lateral_connection_tries_per_unit,
    learning_rate=learning_rate,
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy(),
             tf.keras.metrics.Precision(),
             tf.keras.metrics.Recall()],
    epochs=epochs,
    project_name=f"{PROJECT_NAME}_meta_{meta_trial_number}",
    # use_multiprocessing_for_multiple_neural_networks=False,  # pull this param
    model_graphs='model_graphs',
    batch_size=batch_size,
    meta_trial_number=meta_trial_number)
val_binary_accuracy = cerebros_automl.run_random_search()


No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


nan
>nnf>ceil
k is: 0 value is: [{'1': <class 'cerebros.units.units.InputUnit'>}]
0
k is: 1 value is: [{'3': <class 'cerebros.units.units.DenseUnit'>}]
1
Trying to create level 1
We think level 1's predecessors are: [0]
k is: 2 value is: [{'4': <class 'cerebros.units.units.DenseUnit'>}, {'2': <class 'cerebros.units.units.DenseUnit'>}, {'2': <class 'cerebros.units.units.DenseUnit'>}, {'1': <class 'cerebros.units.units.DenseUnit'>}]
2
Trying to create level 2
We think level 2's predecessors are: [0, 1]
k is: 3 value is: [{'2': <class 'cerebros.units.units.DenseUnit'>}, {'3': <class 'cerebros.units.units.DenseUnit'>}, {'4': <class 'cerebros.units.units.DenseUnit'>}]
3
Trying to create level 3
We think level 3's predecessors are: [0, 1, 2]
k is: 4 value is: [{'1': <class 'cerebros.units.units.FinalDenseUnit'>}]
4
Trying to create Final level 4
Trying to create level 4
We think level final level 4's predecessors are: [0, 1, 2, 3]
levels:
[0, 1, 2, 3, 4]
{'0': 'InputUnitModule'}
{'3': <class

INFO:tensorflow:Assets written to: 2023_01_09_01_03_cerebros_auto_ml_test_meta_776979558490/models/tr_0000000000000000_subtrial_0000000000000000/assets


INFO:tensorflow:Assets written to: 2023_01_09_01_03_cerebros_auto_ml_test_meta_776979558490/models/tr_0000000000000000_subtrial_0000000000000000/assets


returning trial 0 oracles
       loss  binary_accuracy  precision    recall   val_loss  \
0  0.923414         0.452906   0.333333  0.500000  12.975286   
1  0.825317         0.519038   0.384921  0.532967  11.195849   
2  0.758754         0.563126   0.422414  0.538462  10.302246   
3  0.709852         0.603206   0.461905  0.532967   9.456187   
4  0.665995         0.627254   0.489691  0.521978   8.708330   
5  0.633213         0.659319   0.530928  0.565934   7.964751   
6  0.604129         0.701403   0.592179  0.582418   7.241508   
7  0.588480         0.681363   0.564972  0.549451   6.536664   
8  0.564812         0.721443   0.628743  0.576923   5.917686   
9  0.556482         0.713427   0.610169  0.593407   5.382143   

   val_binary_accuracy  val_precision  val_recall  trial_number  \
0             0.319703       0.318352    0.988372             0   
1             0.319703       0.318352    0.988372             0   
2             0.323420       0.320896    1.000000             0   
3

KeyboardInterrupt: 

In [2]:
print(val_binary_accuracy)

1
